# Cylinder Counting - Google Colab Edition

Run this notebook in Google Colab to train the model and count cylinders with the refined visualization.

## 1. Setup Environment
Run this cell to install the necessary libraries.

In [ ]:
!pip install ultralytics
import os
import shutil
import cv2
from ultralytics import YOLO
from google.colab import files

print("Setup complete.")

## 2. Upload Files
Please upload the following files to the Colab environment (drag and drop them into the file browser on the left, or run the cell below to upload):
1. `Annotate Cyinders.v2i.yolov8.zip` (Your dataset)
2. `Task.mp4` (The video to process)

In [ ]:
# Optional: Run this to upload files via button if drag-and-drop doesn't work
# uploaded = files.upload()

## 3. Prepare Dataset & Train
This will unzip your dataset and train the YOLO model.

In [ ]:
import yaml
DATASET_ZIP = "Annotate Cyinders.v2i.yolov8.zip"
DATASET_DIR = "Annotate Cyinders.v2i.yolov8"

# Check if dataset zip exists
if os.path.exists(DATASET_ZIP):
    # Check if already unzipped or data.yaml exists in root
    if not os.path.exists("data.yaml") and not os.path.isdir(DATASET_DIR):
        print("Unzipping dataset...")
        !unzip -q "{DATASET_ZIP}"
        print("Unzip complete.")
    else:
        print("Dataset files found.")
else:
    print("WARNING: Dataset zip file not found! Please upload it.")

# Determine correct path to data.yaml
data_yaml_path = None
if os.path.exists("data.yaml"):
    data_yaml_path = os.path.abspath("data.yaml")
elif os.path.exists(os.path.join(DATASET_DIR, "data.yaml")):
    data_yaml_path = os.path.abspath(os.path.join(DATASET_DIR, "data.yaml"))

if data_yaml_path:
    print(f"Found data.yaml at: {data_yaml_path}")
    
    # Fix paths in data.yaml to be absolute
    with open(data_yaml_path, 'r') as f:
        data = yaml.safe_load(f)
    
    # Update paths to absolute if they are relative
    base_dir = os.path.dirname(data_yaml_path)
    if 'train' in data and not os.path.isabs(data['train']):
        data['train'] = os.path.abspath(os.path.join(base_dir, data['train']))
    if 'val' in data and not os.path.isabs(data['val']):
        data['val'] = os.path.abspath(os.path.join(base_dir, data['val']))
    if 'test' in data and not os.path.isabs(data['test']):
        data['test'] = os.path.abspath(os.path.join(base_dir, data['test']))
        
    with open(data_yaml_path, 'w') as f:
        yaml.dump(data, f)
    print("Updated data.yaml with absolute paths.")

    # Train Model
    print("Starting Training...")
    model = YOLO("yolov8n.pt")
    results = model.train(data=data_yaml_path, epochs=100, imgsz=640)
    print("Training Complete.")
else:
    print("ERROR: data.yaml not found! Check your dataset zip structure.")

## 4. Run Inference (Refined Visualization)
This code applies the light bounding boxes and truck ROI filter.

In [ ]:
VIDEO_PATH = "Task.mp4"
OUTPUT_VIDEO_PATH = "output_tracking_colab.mp4"

def process_video_colab(model_path):
    print(f"Processing video: {VIDEO_PATH}...")
    
    if not os.path.exists(VIDEO_PATH):
        print(f"Error: {VIDEO_PATH} not found. Please upload it.")
        return

    model = YOLO(model_path)
    cap = cv2.VideoCapture(VIDEO_PATH)
    w, h, fps = (int(cap.get(x)) for x in (cv2.CAP_PROP_FRAME_WIDTH, cv2.CAP_PROP_FRAME_HEIGHT, cv2.CAP_PROP_FPS))
    video_writer = cv2.VideoWriter(OUTPUT_VIDEO_PATH, cv2.VideoWriter_fourcc(*"mp4v"), fps, (w, h))

    # --- CONFIGURATION ---
    ZONE_Y_MIN = int(h * 0.40)
    ZONE_Y_MAX = int(h * 0.60)
    TRUCK_ROI_X_MIN = int(w * 0.15) 
    TRUCK_ROI_X_MAX = int(w * 0.85)
    TRUCK_ROI_Y_MIN = int(h * 0.05)
    TRUCK_ROI_Y_MAX = int(h * 0.95)
    counted_ids = set()

    while cap.isOpened():
        success, frame = cap.read()
        if not success: break

        results = model.track(frame, persist=True, verbose=False)
        annotated_frame = frame.copy()

        if results[0].boxes.id is not None:
            boxes = results[0].boxes.xyxy.cpu()
            track_ids = results[0].boxes.id.int().cpu().tolist()

            for box, track_id in zip(boxes, track_ids):
                x1, y1, x2, y2 = box
                cx = int((x1 + x2) / 2)
                cy = int((y1 + y2) / 2)

                if (TRUCK_ROI_X_MIN < cx < TRUCK_ROI_X_MAX) and (TRUCK_ROI_Y_MIN < cy < TRUCK_ROI_Y_MAX):
                    cv2.rectangle(annotated_frame, (int(x1), int(y1)), (int(x2), int(y2)), (255, 255, 200), 1)
                    cv2.putText(annotated_frame, f"{track_id}", (int(x1), int(y1)-5), 
                                cv2.FONT_HERSHEY_SIMPLEX, 0.4, (255, 255, 200), 1)
                    if ZONE_Y_MIN < cy < ZONE_Y_MAX:
                        counted_ids.add(track_id)
        
        # Draw Count
        count_str = f"Total: {len(counted_ids)}"
        (text_w, text_h), _ = cv2.getTextSize(count_str, cv2.FONT_HERSHEY_SIMPLEX, 2.5, 5)
        overlay = annotated_frame.copy()
        cv2.rectangle(overlay, (20, 20), (20 + text_w + 20, 80 + 20), (0, 0, 0), -1)
        cv2.addWeighted(overlay, 0.5, annotated_frame, 0.5, 0, annotated_frame)
        cv2.putText(annotated_frame, count_str, (30, 80), cv2.FONT_HERSHEY_SIMPLEX, 2.5, (0, 255, 255), 5)
        video_writer.write(annotated_frame)
    
    cap.release()
    video_writer.release()
    print(f"Saved to {OUTPUT_VIDEO_PATH}")

# Find best model
import glob
runs = glob.glob("runs/detect/train*")
if runs:
    latest_run = max(runs, key=os.path.getmtime)
    best_model = os.path.join(latest_run, "weights", "best.pt")
    process_video_colab(best_model)
else:
    print("Model not found. Did training finish?")

## 5. Download Result
Run this to download the processed video.

In [ ]:
from google.colab import files
if os.path.exists("output_tracking_colab.mp4"):
    files.download("output_tracking_colab.mp4")
else:
    print("Output video not found.")